In [62]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import torch.nn as nn 
import os
from skimage import io, transform
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
from PIL import Image
import torch.nn.functional as F

In [80]:
num_epochs = 200
batch_size = 4 
learning_rate = 1e-3

In [73]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,48,3,padding=1)

        self.conv2 = nn.Conv2d(48,48,3,padding=1)

        self.pool = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(48,48,3,padding=1)

        self.conv4 = nn.Conv2d(48,48,3,padding=1)


        self.conv5 = nn.Conv2d(48,48,3,padding=1)

        self.conv6 = nn.Conv2d(48,48,3,padding=1)


        self.upscale = nn.Upsample(scale_factor=2)

        self.decov1 = nn.Conv2d(96,96,3,padding=1)


        self.decov2 = nn.Conv2d(96,96,3,padding=1)

        self.decov3 = nn.Conv2d(144,96,3,padding=1)


        self.decov4 = nn.Conv2d(96,96,3,padding=1)


        self.decov5 = nn.Conv2d(144,96,3,padding=1)

        self.decov6 = nn.Conv2d(96,96,3,padding=1)


        self.decov7 = nn.Conv2d(144,96,3,padding=1)

        self.decov8 = nn.Conv2d(96,96,3,padding=1)


        self.decov9 = nn.Conv2d(99,64,3,padding=1)

        self.decov10 = nn.Conv2d(64,32,3,padding=1)

        self.conv7 = nn.Conv2d(32,3,3,padding=1)
        
    def forward(self, x):
        
        skips = [x]
        
        # Enc. Conv0
        x = F.leaky_relu(self.conv1(x))
        
        # Enc. Conv1
        x = F.leaky_relu(self.conv2(x))
        
        #pool1
        x = self.pool(x)
        skips.append(x)
        
        # Enc. Conv2 and p00l2
        x = self.pool(F.leaky_relu(self.conv3(x)))
        
        skips.append(x)
        
        # Enc. Conv3 and p00l3
        x = self.pool(F.leaky_relu(self.conv4(x)))
        
        skips.append(x)
        
        # Enc. Conv4 and pool4
        x = self.pool(F.leaky_relu(self.conv5(x)))
        
        skips.append(x)
        
        # Enc. Conv5 and pool5
        x = self.pool(F.leaky_relu(self.conv6(x)))
        
        # Enc. Conv6
        x = F.leaky_relu(self.conv6(x))
        
        #---------#
        #upsample 5
        x = self.upscale(x)
        
        # concat 5
        x = torch.cat([x, skips.pop()],1)        
        
        x = F.leaky_relu(self.decov1(x))
        
        x = self.upscale(F.leaky_relu_(self.decov2(x)))

        x = torch.cat([x, skips.pop()],1)
        
        x = F.leaky_relu(self.decov3(x))
        x = self.upscale(F.leaky_relu(self.decov4(x)))
        
        # concat 3
        x = torch.cat([x, skips.pop()],1)
        x = F.leaky_relu(self.decov5(x))
        
        x = self.upscale(F.leaky_relu(self.decov6(x)))
        # concat2
        x = torch.cat([x, skips.pop()],1)
        x = F.leaky_relu(self.decov7(x))
        x = self.upscale(F.leaky_relu(self.decov8(x)))
        
        #concat 1
        x = torch.cat([x, skips.pop()],1)
        x = F.leaky_relu(self.decov9(x))
        x = F.leaky_relu(self.decov10(x))
        
        
        x = self.conv7(x)
        
        return x

In [74]:
def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.MSELoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [75]:
import time

In [76]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [77]:
print(device)

cuda:0


In [84]:
def trainNet(net, batch_size, n_epochs, learning_rate):
    
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    train_loader = getTrainLoader(batch_size)
    
    test_loader = getTestLoader(batch_size)
    
    n_batches = len(train_loader)
    
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    training_start_time = time.time()
    
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(zip(train_loader, test_loader)):
            
            
            
            optimizer.zero_grad()
            inputs, labels = data
#             print("input - ",inputs[0].shape)
            inputs = inputs[0].float()
            labels = labels[0].float()
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = net(inputs)
#             print("output = ", outputs.shape)
            
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            running_loss += loss_size.data[0]
            
            total_train_loss += loss_size.data[0]
            
            print(i,end='\r')
            
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()

In [85]:
net = Net().to(device)
# net = Net()
trainNet(net, batch_size=batch_size, n_epochs=num_epochs, learning_rate=learning_rate)

===== HYPERPARAMETERS =====
batch_size= 4
epochs= 200
learning_rate= 0.001


/home/turing/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/home/turing/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel/__main__.py:46: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/turing/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel/__main__.py:48: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 1, 10% 	 train_loss: 27740510.00 took: 657.43s
Epoch 1, 20% 	 train_loss: 76.77 took: 637.86s


KeyboardInterrupt: 

Process Process-16:
Process Process-15:
Process Process-14:
Process Process-13:
Traceback (most recent call last):
  File "/home/turing/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/turing/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/turing/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/turing/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-63-88826ae5edc9>", line 15, in __getitem__
    image = io.imread(img_path)
  File "/home/turing/anaconda3/envs/pytorch/lib/python3.6/site-packages/skimage/io/_io.py", line 62, in imread
    img = call_pl